In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from torchvision import models,transforms
import matplotlib.pyplot as plt
import pickle
from collections import OrderedDict
import csv
import collections
from  PIL import Image
from tqdm.notebook import tqdm_notebook
from scipy.spatial import distance
import warnings
warnings.filterwarnings('ignore')
import math
device = torch.device("mps" if torch.has_mps else "cpu")
print(device)

mps


In [2]:
### load data 
# create df to contain all identities, their image file names, their ethnicities
path = "data/RFW/images/test/txts/"
img_path = 'data/RFW/images/test/data/'

# African images
african_images = pd.read_csv(path + 'African/African_images.txt', sep="\t", header=None)
african_images.columns = ['File', 'Label']
african_images['identityID'] = african_images['File'].str[:-9]
african_images['faceID'] = african_images['File'].str[-8:-4]
african_images['Ethnicity'] = 'African'
# Asian images
asian_images = pd.read_csv(path + 'Asian/Asian_images.txt', sep="\t", header=None)
asian_images.columns = ['File', 'Label']
asian_images['identityID'] = asian_images['File'].str[:-9]
asian_images['faceID'] = asian_images['File'].str[-8:-4]
asian_images['Ethnicity'] = 'Asian'
# Caucasian images
caucasian_images = pd.read_csv(path + 'Caucasian/Caucasian_images.txt', sep="\t", header=None)
caucasian_images.columns = ['File', 'Label']
caucasian_images['identityID'] = caucasian_images['File'].str[:-9]
caucasian_images['faceID'] = caucasian_images['File'].str[-8:-4]
caucasian_images['Ethnicity'] = 'Caucasian'
# Indian images
indian_images = pd.read_csv(path + 'Indian/Indian_images.txt', sep="\t", header=None)
indian_images.columns = ['File', 'Label']
indian_images['identityID'] = indian_images['File'].str[:-9]
indian_images['faceID'] = indian_images['File'].str[-8:-4]
indian_images['Ethnicity'] = 'Indian'
all_images = pd.concat([african_images,asian_images,caucasian_images,indian_images])

# remove any duplicate identities
v = all_images.reset_index().groupby('identityID').Ethnicity.nunique()
dup = v[v>1].index.tolist()
all_images = all_images[~all_images['identityID'].isin(dup)]

# get first image from each identity and use it as reference
identities = np.array(all_images.identityID.unique().tolist()).astype(object)
file_end =  np.array('_0001.jpg'.split()*len(identities)).astype(object)
first_images = identities + file_end

references = all_images[all_images['File'].isin(first_images)]
candidates = all_images[~all_images['File'].isin(first_images)]

In [3]:
african_references = references[references.Ethnicity=='African']
african_candidates = candidates[candidates.Ethnicity=='African']
caucasian_references = references[references.Ethnicity=='Caucasian']
caucasian_candidates = candidates[candidates.Ethnicity=='Caucasian']

In [4]:
__all__ = ['SENet', 'senet50']

def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

# This SEModule is not used.
class SEModule(nn.Module):

    def __init__(self, planes, compress_rate):
        super(SEModule, self).__init__()
        self.conv1 = nn.Conv2d(planes, planes // compress_rate, kernel_size=1, stride=1, bias=True)
        self.conv2 = nn.Conv2d(planes // compress_rate, planes, kernel_size=1, stride=1, bias=True)
        self.relu = nn.ReLU(inplace=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        module_input = x
        x = F.avg_pool2d(module_input, kernel_size=module_input.size(2))
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.sigmoid(x)
        return module_input * x


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

        # SENet
        compress_rate = 16
        # self.se_block = SEModule(planes * 4, compress_rate)  # this is not used.
        self.conv4 = nn.Conv2d(planes * 4, planes * 4 // compress_rate, kernel_size=1, stride=1, bias=True)
        self.conv5 = nn.Conv2d(planes * 4 // compress_rate, planes * 4, kernel_size=1, stride=1, bias=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)


        ## senet
        out2 = F.avg_pool2d(out, kernel_size=out.size(2))
        out2 = self.conv4(out2)
        out2 = self.relu(out2)
        out2 = self.conv5(out2)
        out2 = self.sigmoid(out2)
        # out2 = self.se_block.forward(out)  # not used

        if self.downsample is not None:
            residual = self.downsample(x)

        out = out2 * out + residual
        # out = out2 + residual  # not used
        out = self.relu(out)
        return out


class SENet(nn.Module):

    def __init__(self, block, layers, num_classes=8631, include_top=False):
        self.inplanes = 64
        super(SENet, self).__init__()
        self.include_top = include_top
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=0, ceil_mode=True)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        
        if not self.include_top:
            return x
        
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


def senet50(**kwargs):
    """Constructs a SENet-50 model.
    """
    model = SENet(Bottleneck, [3, 4, 6, 3], **kwargs)
    return model

# create dataset class for RFW
class resnetRFW(data.Dataset):
    
    '''
    This will be a class to load data from RFW for resnet50 model
    '''
     
    mean_bgr = np.array([91.4953, 103.8827, 131.0912])  # from resnet50_ft.prototxt

    def __init__(self,img_path,img_df):
        """
        :param img_path: dataset directory
        :param img_df: contains image file names and other information
        """
        assert os.path.exists(img_path), "root: {} not found.".format(img_path)
        self.img_path = img_path
        self.img_df = img_df
        self.img_info = []

        for i, row in self.img_df.iterrows():
            self.img_info.append({
                'img_file': row.Ethnicity + '/' + row.identityID + '/' + row.File,
                'identityID': row.identityID,
                'Ethnicity': row.Ethnicity,
                'faceID': row.faceID,
            })
            if i % 5000 == 0:
                print("processing: {} images".format(i))

    def __len__(self):
        return len(self.img_info)

    def __getitem__(self, index):
        info = self.img_info[index]
        img_file = info['img_file']
        img = Image.open(os.path.join(self.img_path, img_file))
        img = transforms.Resize(256)(img)
        img = transforms.CenterCrop(224)(img)
        img = np.array(img, dtype=np.uint8)
        assert len(img.shape) == 3  # assumes color images and no alpha channel

        Ethnicity = info['Ethnicity']
        identityID = info['identityID']
        faceID = info['faceID']
        return self.transform(img), identityID, Ethnicity, faceID
  

    def transform(self, img):
        img = img[:, :, ::-1]  # RGB -> BGR
        img = img.astype(np.float32)
        img -= self.mean_bgr
        img = img.transpose(2, 0, 1)  # C x H x W
        img = torch.from_numpy(img).float()
        return img

    def untransform(self, img, lbl):
        img = img.numpy()
        img = img.transpose(1, 2, 0)
        #img += self.mean_bgr
        img = img.astype(np.uint8)
        img = img[:, :, ::-1]
        return img, lbl
        
def apply_model(model,dataloader,file_prefix,device):
    model.eval()
    outputs = []
    identities = []
    ethnicities = []
    faceIDs = []
    with torch.no_grad():
        for i, (imgs, identityID, Ethnicity, faceID) in tqdm_notebook(enumerate(dataloader),total=len(dataloader)):
            imgs = imgs.to(device)
            x = model(imgs)
            out = x.view(x.size(0),-1)
            outputs.append(out)
            identities.append(np.array(identityID))
            ethnicities.append(np.array(Ethnicity))
            faceIDs.append(np.array(faceID))

    outputs=torch.cat(outputs)
    identities= np.concatenate(np.array(identities)).ravel()
    ethnicities= np.concatenate(np.array(ethnicities)).ravel()
    faceIDs= np.concatenate(np.array(faceIDs)).ravel()

    torch.save(outputs, file_prefix + '_outputs.pt')
    np.save(file_prefix + '_identities.npy', identities)
    np.save(file_prefix + '_ethnicities.npy', ethnicities)
    np.save(file_prefix + '_faceIDs.npy', faceIDs)
    return outputs, identities, ethnicities, faceIDs

In [5]:
# load model and assign weights
ft_weights="weights/senet50_ft_weight.pkl"
senet50_ft = senet50()

with open(ft_weights, 'rb') as f:
    weights = pickle.load(f, encoding='latin1')

own_state = senet50_ft.state_dict()
for name, param in weights.items():
    if name in own_state:
        try:
            own_state[name].copy_(torch.from_numpy(param))
        except Exception:
            raise RuntimeError('While copying the parameter named {}, whose dimensions in the model are {} and whose '\
                                'dimensions in the checkpoint are {}.'.format(name, own_state[name].size(), param.shape))
    else:
        raise KeyError('unexpected key "{}" in state_dict'.format(name))

model_ft = senet50_ft.to(device=device)

In [6]:
kwargs = {'num_workers': 4, 'pin_memory': True} if torch.cuda.is_available() else {}
# load reference images
african_reference_dataset = resnetRFW(img_path,african_references.reset_index(drop=True))
african_reference_loader = torch.utils.data.DataLoader(african_reference_dataset, batch_size=4, shuffle=False, **kwargs)
# load candidate images
african_candidate_dataset = resnetRFW(img_path,african_candidates.reset_index(drop=True))
african_candidate_loader = torch.utils.data.DataLoader(african_candidate_dataset, batch_size=4, shuffle=False, **kwargs)

processing: 0 images
processing: 0 images
processing: 5000 images


In [7]:
african_reference_outputs, african_reference_identities, african_reference_ethnicities, african_reference_faceIDs = apply_model(model_ft,african_reference_loader,'outputs/RFW/ft/reference2',device)
african_candidate_outputs, african_candidate_identities, african_candidate_ethnicities, african_candidate_faceIDs = apply_model(model_ft,african_candidate_loader,'outputs/RFW/ft/candidate2',device)

  0%|          | 0/746 [00:00<?, ?it/s]

  0%|          | 0/1849 [00:00<?, ?it/s]

In [8]:
def corr2_coeff(A, B):
    # Rowwise mean of input arrays & subtract from input arrays themeselves
    A_mA = A - A.mean(1)[:, None]
    B_mB = B - B.mean(1)[:, None]

    # Sum of squares across rows
    ssA = (A_mA**2).sum(1)
    ssB = (B_mB**2).sum(1)

    # Finally get corr coeff
    return torch.matmul(A_mA, B_mB.T) / torch.sqrt(torch.matmul(ssA[:, None],ssB[None]))
def cos_sim(a, b, eps=1e-8):
    """
    added eps for numerical stability
    """
    a_n, b_n = a.norm(dim=1)[:, None], b.norm(dim=1)[:, None]
    a_norm = a / torch.max(a_n, eps * torch.ones_like(a_n))
    b_norm = b / torch.max(b_n, eps * torch.ones_like(b_n))
    sim_mt = torch.mm(a_norm, b_norm.transpose(0, 1))
    return sim_mt

In [9]:
cor = corr2_coeff(african_reference_outputs,african_candidate_outputs).cpu().detach().numpy()
cos = cos_sim(african_reference_outputs,african_candidate_outputs).cpu().detach().numpy()

In [10]:
def verification(similarity_mat, thresh,reference_identities,reference_ethnicities,candidate_identities):
    verification = pd.DataFrame(columns=['reference_identity','reference_ethnicity','TPR','TNR','FPR','FNR'])
    match_mat = np.abs(similarity_mat)>=thresh
    for i, match_row in tqdm_notebook(enumerate(match_mat),total=len(match_mat)):
        identity = reference_identities[i]
        ethnicity = reference_ethnicities[i]
        matches = candidate_identities[match_row]  
        not_matches = candidate_identities[~match_row]
        TP = np.sum(matches == identity)
        FP = np.sum(matches != identity)
        TN = np.sum(not_matches != identity)
        FN = np.sum(not_matches == identity)
        TPR = TP/(TP+FN)
        TNR = TN/(TN+FP)
        FPR = FP/(TN+FP)
        FNR = FN/(TP+FN)
        row = {'reference_identity': identity,
           'reference_ethnicity': ethnicity, 
           'TPR': TPR,
           'TNR': TNR,
           'FPR': FPR,
           'FNR': FNR}
        verification = verification.append(row,ignore_index=True)
    return verification

In [13]:
for thresh in np.arange(0.59,0.61,0.0025):
    cos_verificaiton = verification(cos, thresh,african_reference_identities,african_reference_ethnicities,african_candidate_identities)
    TPR = np.round(cos_verificaiton.mean(axis=0)['TPR'],4)
    FPR = np.round(cos_verificaiton.mean(axis=0)['FPR'],4)


    print('threshold =', np.round(thresh,4), 'TPR =', TPR)
    print('threshold =', np.round(thresh,4), 'FPR =', FPR)
   


  0%|          | 0/2982 [00:00<?, ?it/s]

threshold = 0.59 TPR = 0.8486
threshold = 0.59 FPR = 0.0118


  0%|          | 0/2982 [00:00<?, ?it/s]

threshold = 0.5925 TPR = 0.8432
threshold = 0.5925 FPR = 0.0111


  0%|          | 0/2982 [00:00<?, ?it/s]

...

threshold = 0.5975 TPR = 0.8326
threshold = 0.5975 FPR = 0.0097


  0%|          | 0/2982 [00:00<?, ?it/s]

threshold = 0.6 TPR = 0.829
threshold = 0.6 FPR = 0.0091


  0%|          | 0/2982 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [14]:
# perform on caucasian images
kwargs = {'num_workers': 4, 'pin_memory': True} if torch.cuda.is_available() else {}
# load reference images
caucasian_reference_dataset = resnetRFW(img_path,caucasian_references.reset_index(drop=True))
caucasian_reference_loader = torch.utils.data.DataLoader(caucasian_reference_dataset, batch_size=4, shuffle=False, **kwargs)
# load candidate images
caucasian_candidate_dataset = resnetRFW(img_path,caucasian_candidates.reset_index(drop=True))
caucasian_candidate_loader = torch.utils.data.DataLoader(caucasian_candidate_dataset, batch_size=4, shuffle=False, **kwargs)

caucasian_reference_outputs, caucasian_reference_identities, caucasian_reference_ethnicities, caucasian_reference_faceIDs = apply_model(model_ft,caucasian_reference_loader,'outputs/RFW/ft/reference2_caucasian',device)
caucasian_candidate_outputs, caucasian_candidate_identities, caucasian_candidate_ethnicities, caucasian_candidate_faceIDs = apply_model(model_ft,caucasian_candidate_loader,'outputs/RFW/ft/candidate2_caucasian',device)

processing: 0 images
processing: 0 images
processing: 5000 images


  0%|          | 0/740 [00:00<?, ?it/s]

  0%|          | 0/1810 [00:00<?, ?it/s]

In [15]:
caucasian_cor = corr2_coeff(caucasian_reference_outputs,caucasian_candidate_outputs).cpu().detach().numpy()
caucasian_cos = cos_sim(caucasian_reference_outputs,caucasian_candidate_outputs).cpu().detach().numpy()

In [18]:
for thresh in np.arange(0.475,0.55,0.0025):
    cos_verificaiton = verification(caucasian_cos, thresh,caucasian_reference_identities,caucasian_reference_ethnicities,caucasian_candidate_identities)
    TPR = np.round(cos_verificaiton.mean(axis=0)['TPR'],4)
    FPR = np.round(cos_verificaiton.mean(axis=0)['FPR'],4)


    print('threshold =', np.round(thresh,4), 'TPR =', TPR)
    print('threshold =', np.round(thresh,4), 'FPR =', FPR)

  0%|          | 0/2958 [00:00<?, ?it/s]

threshold = 0.475 TPR = 0.9426
threshold = 0.475 FPR = 0.0137


  0%|          | 0/2958 [00:00<?, ?it/s]

threshold = 0.4775 TPR = 0.9399
threshold = 0.4775 FPR = 0.0129


  0%|          | 0/2958 [00:00<?, ?it/s]

...

threshold = 0.4875 TPR = 0.93
threshold = 0.4875 FPR = 0.0102


  0%|          | 0/2958 [00:00<?, ?it/s]

threshold = 0.49 TPR = 0.9277
threshold = 0.49 FPR = 0.0095


  0%|          | 0/2958 [00:00<?, ?it/s]

KeyboardInterrupt: 